This is some connect 

In [34]:
# Must be included at the beginning of each new notebook. Remember to change the app name.
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basics').getOrCreate()

In [35]:
file = 'violence_data_NEW.csv'
df = spark.read.csv(file, header=True, inferSchema=True)

In [36]:
from pyspark.sql.types import (StructField,StringType,IntegerType,StructType)

In [37]:
#3.1 drop the 'Usere feedback' field
columns_to_drop = ['User feedback']
new_df = df.drop(*columns_to_drop)

In [38]:
new_df.columns

['RecordID',
 'Country',
 'Gender',
 'Demographics Question',
 'Demographics Response',
 'Question',
 'Survey Year',
 'Value']

In [39]:
#use your sales average to fill missing data. 
from pyspark.sql.functions import mean
#3.2 Dealing with the null value in 'Value' field
mean_value = df.select(mean(df['Value'])).collect()[0][0]
new_df = new_df.na.fill({'Value': mean_value})

In [40]:
new_df.show()
new_df.printSchema()

+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----------------+
|RecordID|    Country|Gender|Demographics Question|Demographics Response|            Question|Survey Year|            Value|
+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----------------+
|       1|Afghanistan|     F|       Marital status|        Never married|... if she burns ...| 01/01/2015|19.76253687315632|
|       1|Afghanistan|     F|            Education|               Higher|... if she burns ...| 01/01/2015|             10.1|
|       1|Afghanistan|     F|            Education|            Secondary|... if she burns ...| 01/01/2015|             13.7|
|       1|Afghanistan|     F|            Education|              Primary|... if she burns ...| 01/01/2015|             13.8|
|       1|Afghanistan|     F|       Marital status| Widowed, divorced...|... if she burns ...| 01/01/2015|             13.8|


In [41]:
new = 'Value'

new_df.select(new).show()
new_df.select(new).describe().show()

+-----------------+
|            Value|
+-----------------+
|19.76253687315632|
|             10.1|
|             13.7|
|             13.8|
|             13.8|
|             17.0|
|             17.3|
|             18.0|
|             18.1|
|             18.2|
|             18.3|
|             18.3|
|             18.8|
|             19.1|
|             20.8|
|19.76253687315632|
|              4.5|
|              4.6|
|              5.2|
|              6.3|
+-----------------+
only showing top 20 rows

+-------+------------------+
|summary|             Value|
+-------+------------------+
|  count|             12600|
|   mean| 19.76253687315705|
| stddev|16.005588776096097|
|    min|               0.0|
|    max|              86.9|
+-------+------------------+



In [42]:
#3.3 Constructing new data from 'Value' called 'Measurement'
from pyspark.sql.functions import mean, col, when 
data_copy_33 = new_df.withColumn(
    'Measurement',
    when(col('Value') == mean_value, 0).otherwise(1)
)


In [43]:
new = 'Measurement'

data_copy_33.select(new).show()
data_copy_33.select(new).describe().show()

+-----------+
|Measurement|
+-----------+
|          0|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          0|
|          1|
|          1|
|          1|
|          1|
+-----------+
only showing top 20 rows

+-------+-------------------+
|summary|        Measurement|
+-------+-------------------+
|  count|              12600|
|   mean| 0.8878571428571429|
| stddev|0.31555465373180014|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [44]:
data_copy_33.printSchema()

root
 |-- RecordID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Demographics Question: string (nullable = true)
 |-- Demographics Response: string (nullable = true)
 |-- Question: string (nullable = true)
 |-- Survey Year: string (nullable = true)
 |-- Value: double (nullable = false)
 |-- Measurement: integer (nullable = false)



In [45]:
#3.4 Merging with other dataframe
file2 = 'violence_data_merge.csv'
merge_df = spark.read.csv(file2, header=True, inferSchema=True)

In [46]:
merge_df.show()

+--------+----+----+----+----+----+----+----+-------------+
|RecordID| _c1| _c2| _c3| _c4| _c5| _c6| _c7|User feedback|
+--------+----+----+----+----+----+----+----+-------------+
|       1|null|null|null|null|null|null|null|         null|
|       1|null|null|null|null|null|null|null|         null|
|       1|null|null|null|null|null|null|null|         null|
|       1|null|null|null|null|null|null|null|           10|
|       1|null|null|null|null|null|null|null|         null|
|       1|null|null|null|null|null|null|null|         null|
|       1|null|null|null|null|null|null|null|           23|
|       1|null|null|null|null|null|null|null|            e|
|       1|null|null|null|null|null|null|null|           4t|
|       1|null|null|null|null|null|null|null|             |
|       1|null|null|null|null|null|null|null|         null|
|       1|null|null|null|null|null|null|null|         null|
|       1|null|null|null|null|null|null|null|          532|
|       1|null|null|null|null|null|null|

23/10/11 02:25:32 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: RecordID, , , , , , , , User feedback
 Schema: RecordID, _c1, _c2, _c3, _c4, _c5, _c6, _c7, User feedback
Expected: _c1 but found: 
CSV file: file:///home/ubuntu/722_I4/violence_data_merge.csv


In [47]:
merged_data = data_copy_33.join(merge_df, 'RecordID')

In [48]:
merged_data.show()
merged_data.printSchema()

+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----------------+-----------+----+----+----+----+----+----+----+-------------+
|RecordID|    Country|Gender|Demographics Question|Demographics Response|            Question|Survey Year|            Value|Measurement| _c1| _c2| _c3| _c4| _c5| _c6| _c7|User feedback|
+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----------------+-----------+----+----+----+----+----+----+----+-------------+
|       1|Afghanistan|     F|       Marital status|        Never married|... if she burns ...| 01/01/2015|19.76253687315632|          0|null|null|null|null|null|null|null|           4e|
|       1|Afghanistan|     F|       Marital status|        Never married|... if she burns ...| 01/01/2015|19.76253687315632|          0|null|null|null|null|null|null|null|         gf43|
|       1|Afghanistan|     F|       Marital status|        Never marri

23/10/11 02:25:32 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: RecordID, , , , , , , , User feedback
 Schema: RecordID, _c1, _c2, _c3, _c4, _c5, _c6, _c7, User feedback
Expected: _c1 but found: 
CSV file: file:///home/ubuntu/722_I4/violence_data_merge.csv


In [49]:
#3.5 Removing all the other fields that do not have anything contains
columns_to_drop = ['_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7']
merged_data = merged_data.drop(*columns_to_drop)

In [50]:
merged_data.printSchema()
new = 'Value'

df.select(new).show()
merged_data.select(new).describe().show()

root
 |-- RecordID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Demographics Question: string (nullable = true)
 |-- Demographics Response: string (nullable = true)
 |-- Question: string (nullable = true)
 |-- Survey Year: string (nullable = true)
 |-- Value: double (nullable = false)
 |-- Measurement: integer (nullable = false)
 |-- User feedback: string (nullable = true)

+-----+
|Value|
+-----+
| null|
| 10.1|
| 13.7|
| 13.8|
| 13.8|
| 17.0|
| 17.3|
| 18.0|
| 18.1|
| 18.2|
| 18.3|
| 18.3|
| 18.8|
| 19.1|
| 20.8|
| null|
|  4.5|
|  4.6|
|  5.2|
|  6.3|
+-----+
only showing top 20 rows

+-------+------------------+
|summary|             Value|
+-------+------------------+
|  count|            378000|
|   mean| 19.76253687315055|
| stddev|16.004974791697258|
|    min|               0.0|
|    max|              86.9|
+-------+------------------+



In [51]:
#4.1 Feature selection & Encoding the string fields
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.feature import VectorAssembler

In [52]:
#Encoding 'Country'
indexer = StringIndexer(inputCol="Country", outputCol="CountryIndex")
encoder = OneHotEncoder(inputCols=["CountryIndex"], outputCols=["CountryVec"])
df = indexer.fit(df).transform(df)
df = encoder.fit(df).transform(df)

In [53]:
df.printSchema()

root
 |-- RecordID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Demographics Question: string (nullable = true)
 |-- Demographics Response: string (nullable = true)
 |-- Question: string (nullable = true)
 |-- Survey Year: string (nullable = true)
 |-- Value: double (nullable = true)
 |-- User feedback: string (nullable = true)
 |-- CountryIndex: double (nullable = false)
 |-- CountryVec: vector (nullable = true)



In [54]:
#Encoding 'Gender'
indexer = StringIndexer(inputCol="Gender", outputCol="GenderIndex")
encoder = OneHotEncoder(inputCols=["GenderIndex"], outputCols=["GenderVec"])
df = indexer.fit(df).transform(df)
df = encoder.fit(df).transform(df)
df.printSchema()

root
 |-- RecordID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Demographics Question: string (nullable = true)
 |-- Demographics Response: string (nullable = true)
 |-- Question: string (nullable = true)
 |-- Survey Year: string (nullable = true)
 |-- Value: double (nullable = true)
 |-- User feedback: string (nullable = true)
 |-- CountryIndex: double (nullable = false)
 |-- CountryVec: vector (nullable = true)
 |-- GenderIndex: double (nullable = false)
 |-- GenderVec: vector (nullable = true)



In [55]:
#Encoding 'Demographics Response' 
#responseIndexer = StringIndexer(inputCol="Demographics Response", outputCol="ResponseIndex")
#responseEncoder = OneHotEncoder(inputCols=["ResponseIndex"], outputCols=["ResponseVec"])
#df = responseIndexer.fit(df).transform(df)
#df = responseEncoder.fit(df).transform(df)
indexer = StringIndexer(inputCol="Demographics Response", outputCol="ResponseIndex")
df = indexer.fit(df).transform(df)
df.printSchema()

root
 |-- RecordID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Demographics Question: string (nullable = true)
 |-- Demographics Response: string (nullable = true)
 |-- Question: string (nullable = true)
 |-- Survey Year: string (nullable = true)
 |-- Value: double (nullable = true)
 |-- User feedback: string (nullable = true)
 |-- CountryIndex: double (nullable = false)
 |-- CountryVec: vector (nullable = true)
 |-- GenderIndex: double (nullable = false)
 |-- GenderVec: vector (nullable = true)
 |-- ResponseIndex: double (nullable = false)



In [56]:
df.show()
#ret

+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----+-------------+------------+--------------+-----------+-------------+-------------+
|RecordID|    Country|Gender|Demographics Question|Demographics Response|            Question|Survey Year|Value|User feedback|CountryIndex|    CountryVec|GenderIndex|    GenderVec|ResponseIndex|
+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----+-------------+------------+--------------+-----------+-------------+-------------+
|       1|Afghanistan|     F|       Marital status|        Never married|... if she burns ...| 01/01/2015| null|         null|         0.0|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|          7.0|
|       1|Afghanistan|     F|            Education|               Higher|... if she burns ...| 01/01/2015| 10.1|         null|         0.0|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|          5.0|
|       1|Afghanistan|   

In [57]:
#
selectedCols = [col for col in df.columns if col  in ['RecordID','CountryVec','GenderVec',
                                                         'ResponseIndex','GenderIndex']]
new_df = df.select(selectedCols)
new_df.printSchema()

root
 |-- RecordID: integer (nullable = true)
 |-- CountryVec: vector (nullable = true)
 |-- GenderIndex: double (nullable = false)
 |-- GenderVec: vector (nullable = true)
 |-- ResponseIndex: double (nullable = false)



In [58]:
new_df.show()

+--------+--------------+-----------+-------------+-------------+
|RecordID|    CountryVec|GenderIndex|    GenderVec|ResponseIndex|
+--------+--------------+-----------+-------------+-------------+
|       1|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|          7.0|
|       1|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|          5.0|
|       1|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|         11.0|
|       1|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|          9.0|
|       1|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|         14.0|
|       1|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|          4.0|
|       1|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|          0.0|
|       1|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|         12.0|
|       1|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|         10.0|
|       1|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|          1.0|
|       1|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|          6.0|
|       1|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|         13.0|
|       1|

In [59]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import ChiSqSelector, VectorAssembler, StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType


In [60]:
#creating new field which is called 'featuresnew'
feature_columns = [col for col in new_df.columns if (col != 'GenderIndex' or col != 'Value')]  
assembler = VectorAssembler(inputCols=feature_columns, outputCol="featuresnew")
df = assembler.transform(new_df)


In [61]:

from pyspark.ml.feature import ChiSqSelector

selector = ChiSqSelector(numTopFeatures=5, featuresCol='featuresnew',
                        outputCol='selectedFeatures', labelCol='GenderIndex')







In [62]:
df.describe().show()

+-------+------------------+------------------+-----------------+
|summary|          RecordID|       GenderIndex|    ResponseIndex|
+-------+------------------+------------------+-----------------+
|  count|             12600|             12600|            12600|
|   mean|             210.5|               0.5|              7.0|
| stddev|121.24802439346753|0.5000198424509481|4.320665257311134|
|    min|                 1|               0.0|              0.0|
|    max|               420|               1.0|             14.0|
+-------+------------------+------------------+-----------------+



In [63]:
selected_data = selector.fit(df).transform(df)

In [64]:
selected_data.select('selectedFeatures').show()

+--------------------+
|    selectedFeatures|
+--------------------+
|(5,[0,1,4],[1.0,1...|
|(5,[0,1,4],[1.0,1...|
|(5,[0,1,4],[1.0,1...|
|(5,[0,1,4],[1.0,1...|
|(5,[0,1,4],[1.0,1...|
|(5,[0,1,4],[1.0,1...|
|(5,[0,1,4],[1.0,1...|
|(5,[0,1,4],[1.0,1...|
|(5,[0,1,4],[1.0,1...|
|(5,[0,1,4],[1.0,1...|
|(5,[0,1,4],[1.0,1...|
|(5,[0,1,4],[1.0,1...|
|(5,[0,1,4],[1.0,1...|
|(5,[0,1,4],[1.0,1...|
|(5,[0,1,4],[1.0,1...|
|(5,[0,1,3],[1.0,1...|
|(5,[0,1,3],[1.0,1...|
|(5,[0,1,3],[1.0,1...|
|(5,[0,1,3],[1.0,1...|
|(5,[0,1,3],[1.0,1...|
+--------------------+
only showing top 20 rows



In [65]:
selected_data.show()

+--------+--------------+-----------+-------------+-------------+--------------------+--------------------+
|RecordID|    CountryVec|GenderIndex|    GenderVec|ResponseIndex|         featuresnew|    selectedFeatures|
+--------+--------------+-----------+-------------+-------------+--------------------+--------------------+
|       1|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|          7.0|(73,[0,1,71,72],[...|(5,[0,1,4],[1.0,1...|
|       1|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|          5.0|(73,[0,1,71,72],[...|(5,[0,1,4],[1.0,1...|
|       1|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|         11.0|(73,[0,1,71,72],[...|(5,[0,1,4],[1.0,1...|
|       1|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|          9.0|(73,[0,1,71,72],[...|(5,[0,1,4],[1.0,1...|
|       1|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|         14.0|(73,[0,1,71,72],[...|(5,[0,1,4],[1.0,1...|
|       1|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|          4.0|(73,[0,1,71,72],[...|(5,[0,1,4],[1.0,1...|
|       1|(69,[0],[1.0])|   